# CSV Format.

*Juan Sebastián Caballero Roa*.
## CSV Description



Comma Separated Values. This format requires that each element of our set is in oneline, and within this line, each element attribute is separateb by a comma.

The first line of each File is called *header*, it does not contain data, just contain the attribute information.

For example:

>"Name", "Physics", "Math", "Chemistry"
>
>"Ralph", "7.5", "8.2", "9.5"
>
>"Coraline", "10", "9.8", "9"
>
>"Sandra", "7", "9.4", "8.3"

## Reading a CSV file using Python
We will open "Grants 2014. Associations educational field" in https://datos.gob.es/es/catalogo/l01280066-subvenciones-2014-asociaciones-ambito-educativo that is part of the Open data initiative of the Government of Spain:

In [12]:
import csv

The main structure of the CSV that we're going to use is:


>"Asociación","Actividad Subvencionada ","Importe" ("Association","Subsidized Activity","Amount")
>
>"AMPA ANTONIO MACHADO","TALLER FIESTA DE CARNAVAL","94.56"
>
>"AMPA ANTONIO MACHADO","TALLER DIA DEL PADRE","39.04"

The next step is to open the file. We need to download it and bear in mind the rute of the file in or file manager. We can open the file using the function `open`, and, in order to read it we can use the function `csv.reader`. The reader is going to be an iterable, his elements are lists, each list is a line of the CSV file. The first list is the header, and each element of the list is the attribute information.

As it follows, we are going to implement a code in order to calculate the total amount of grants in 2014:

In [11]:
with open('./subvenciones-educacion.csv', encoding='latin1') as file_csv:
    reader = csv.reader(file_csv)
    next(reader, None) #Skips the header
    totalAmount = 0
    for line in reader:
        amountStr = line[2]
        amount = float(amountStr)
        totalAmount += amount
    print(totalAmount)

66487.94


Here are some considerations of the code: 

- We use `encoding='latin1'` due to the fact that the CSV file is based on Spain data so we will have special characters.
- It is necessary to convert the amount string to a float in order to work with numbers.
- We need to skip the header, that's why we use `next(reader, None)`.

## DictReader
We just have use the `reader` function, but this code is not intuitive, using the `DictReader` method, we can have a readable code:

In [14]:
with open('./subvenciones-educacion.csv', encoding='latin1') as file_csv:
    dict_reader = csv.DictReader(file_csv)
    asocs = {}
    for line in dict_reader:
        center = line['Asociación']
        amount = float(line['Importe'])
        if center in asocs:
            asocs[center] += amount
        else:
            asocs[center] = amount
    print(asocs)

{'AMPA ANTONIO MACHADO': 2344.99, 'AMPA BACHILLER ALONSO LOPEZ': 3200.0, 'AMPA CASTILLA': 2604.44, 'AMPA DAOIZ Y VELARDE': 3152.74, 'AMPA EMILIO CASADO': 3015.67, 'AMPA FEDERICO GARCIA LORCA': 1919.06, 'AMPA GABRIEL Y GALAN': 2741.51, 'AMPA LUIS BUÑUEL': 2081.0, 'AMPA MIGUEL HERNANDEZ': 2923.35, 'AMPA MIRAFLORES': 2787.21, 'AMPA PARQUE CATALUÑA': 2604.44, 'AMPA PROFESOR TIERNO GALVÁN': 1286.0, 'AMPA SEIS DE DICIEMBRE': 1950.0, 'AMPA VALDEPALITOS': 3929.5, 'AMPA LA CHOPERA': 1430.0, 'AMPA EL CUQUILLO': 1507.83, 'AMPA VALDELAPARRA': 2465.0, 'AMPA RIVENDEL': 2200.0, 'AMPA AGORA': 2421.67, 'AMPA ALDEBARAN': 3107.05, 'AMPA GINER DE LOS RIOS': 2058.0, 'AMPA SEVERO OCHOA': 3563.9700000000003, 'AMPA VIRGEN DE LA PAZ': 1416.45, 'AMPA JUAN XXIII': 1781.98, 'AMPA SAN ANTONIO': 2101.83, 'AMPA PADRE  MANYANET': 2695.82, 'AMPA FAPA': 3198.43}


We have a dictionary where the key is the name of the association and the value is the total amount that was given from the government to this association. Here we need to clarify how the DictReader structure works:
- DictReader returns a iterable of dictionaries, each dictionary has as keys the attributes of the header, and as values the information of the attribute is the line. 
- Is not necessary to skip the first line due to DictReader does it by himself.
- If we are working with a File that doesn't have a header, we can skip this problem clarifying the attributes using the parameter fieldnames like this: `dict_reader = csv.DictReader(file_csv, fieldnames=["Asociación","Actividad Subvencionada ","Importe" ])`.

## Writing a CSV file using Python
Now that we have loaded and read the file, we can begin to manipulate it. And analogously to how you read it, you have the objects `wirter()` and `DictWriter()`.

We have the functions `writerow(row)` and `writerows()`, where a row refers to the list of the current line
- If we work with object `wirter()`, each row is an iterable of strings and numbers.
- If we work with object `DictWriter()`, each row is a dictionary which values are strings and numbers whose keys are the attributes of the header.

The `DictWriter()` objects allow another functionality, the `writeheader()` function. this function writes in the file the header using the parameter `fieldnames`.

We will add two more attributes to our grant file. This attributes will be "Justificación requerida" (it means justification required) and "Exceso del Importe" (Amount excess). The first will be "Si" (Yes) if the amount is greater than 300 or No otherwise. The second new attribute will be the amount that exceeds 300.

In [16]:
path1 = './subvenciones-educacion.csv'
path2 = './subvenciones-educacion2.csv'
with open(path1, encoding='latin1') as file_read, open(path2,'w' ,encoding='latin1') as file_write:
    dict_reader = csv.DictReader(file_read)
    attributes = dict_reader.fieldnames + ['Justificación requerida', 'Exceso del Importe']
    writer = csv.DictWriter(file_write, fieldnames=attributes)
    writer.writeheader()
    for line in dict_reader:
        if float(line['Importe']) > 300:
            line['Justificación requerida'] = "Si"
            line['Exceso del Importe'] = float(line['Importe']) - 300
        else:
            line['Justificación requerida'] = "No"
            line['Exceso del Importe'] = 0
        writer.writerow(line)

Let's see the details of the code:
1. We open two filesm the first was is the one that we downloaded from the Spanish source, the second one is a blank file that we are going to fill. That's why we need to have the two paths in the file system.
2. First we open the file that we are going to read as we have see in the last examples, but, we open the blank file with `w` parameter, this means that this fille will be Written.
3. We use the fieldnames that the original file has and append the new attributes that we want using list operations.
4. We use the `writeheader()` function to apply the fieldname that we have into the writable file.
5. After fullfilling the dictionaries in the conditional, we have to write each line in the writable file, that's why we use the function `writerow(line)` after the conditional

We just have see the basic aspect of the `csv library`